<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><center><b>Predicting Outdoor Equipment Purchases<br><br> with Data Science Experience Local<br><br> on the IBM Integrated Analytics System</b></center></font></th>
   </tr>
   <tr style="border: none">
      <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/product-line-prediction/images/products_graphics.png?raw=true" alt="Icon"> </th>
   </tr>
</table>

This notebook provides steps and code to load data from the IBM Integrated Analytics System (IIAS), explore this data, create a predictive model, deploy and start scoring with new data. In addition, it introduces basic data cleansing and exploration, pipeline creation, model training, model persistence, model deployment, predictions and scoring.

Some familiarity with Python is helpful. This notebook uses Python 2.0 and Apache® Spark 2.0.

The data is publically available and contains anonymous outdoor equipment purchases.  It has already been loaded on IIAS but can also be downloaded here [GoSales Transactions](https://apsportal.ibm.com/exchange-api/v1/entries/8044492073eb964f46597b4be06ff5ea/data?accessKey=9561295fa407698694b1e254d0099600).  Based on this data, interest in product lines (golf accessories, camping equipment, etc.) will be predicted based on a consumer's gender, age, marital status and job.


## Learning goals

The learning goals of this notebook are:

-  Load IIAS data into an Apache® Spark DataFrame.
-  Explore data.
-  Prepare data for training and evaluation.
-  Create an Apache® Spark machine learning pipeline.
-  Train and evaluate a model.
-  Persist a pipeline and model.
-  Deploy a model for online scoring.
-  Score with sample data.
-  Explore and visualize prediction results using the plotly package.


## Contents

This notebook contains the following parts:

1.	[Load and explore data](#load)
2.	[Create spark ml model](#model)
3.	[Persist model](#persistence)
4.	[Predict locally and visualize](#visualization)
5.	[Deploy and score](#scoring)
7.	[Summary and next steps](#summary)

<a id="load"></a>
## 1. Load and explore data

In this section you will load the data as an Apache® Spark DataFrame and perform basic exploration.  First, a SparkSession will be created.

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

The GoSales data on IIAS will be loaded into a Spark dataframe.

In [ ]:
df_data = spark.read.format('jdbc').options(
          url='jdbc:db2://9.30.106.50:50000/BLUDB',
          driver='com.ibm.db2.jcc.DB2Driver',
          dbtable='db2inst1.GOSALES',
          user='dkikuchi',
          password='Sailfishin2017!').load()


Explore the loaded data by using the following Apache® Spark DataFrame methods:
-  print schema
-  print top ten records
-  count all records

In [ ]:
df_data.printSchema()

As you can see, the data contains five fields. PRODUCT_LINE field is the one we would like to predict (label).

In [ ]:
df_data.show()

In [ ]:
df_data.count()

As you can see, the data set contains 60252 records.

<a id="model"></a>
## 2. Create an Apache® Spark machine learning model

In this section you will learn how to prepare data, create an Apache® Spark machine learning pipeline, and train a model.

### 2.1: Prepare data

In this subsection you will split your data into train, test and predict datasets.

In [ ]:
splitted_data = df_data.randomSplit([0.8, 0.18, 0.02], 24)
train_data = splitted_data[0]
test_data = splitted_data[1]
predict_data = splitted_data[2]

print "Number of training records: " + str(train_data.count())
print "Number of testing records : " + str(test_data.count())
print "Number of prediction records : " + str(predict_data.count())

As you can see our data has been successfully split into three datasets: 

-  The train data set, the largest group, is used for model training.
-  The test data set will be used for model evaluation.
-  The predict data set will be used for predictions.

### 2.2: Create pipeline and train a model

In this section you will create an Apache® Spark machine learning pipeline and then train the model.

In the first step you need to import the Apache® Spark machine learning packages that will be needed in the subsequent steps.

In [ ]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In the following step, convert all the string fields to numeric ones by using the StringIndexer transformer.  This encodes a string column to a column of indices that the machine learning algorithm can consume and evaluate.

In [ ]:
stringIndexer_label = StringIndexer(inputCol="PRODUCT_LINE", outputCol="label").fit(df_data)
stringIndexer_prof = StringIndexer(inputCol="PROFESSION", outputCol="PROFESSION_IX")
stringIndexer_gend = StringIndexer(inputCol="GENDER", outputCol="GENDER_IX")
stringIndexer_mar = StringIndexer(inputCol="MARITAL_STATUS", outputCol="MARITAL_STATUS_IX")

In the following step, create a feature vector by combining all features together.

In [ ]:
vectorAssembler_features = VectorAssembler(inputCols=["GENDER_IX", "AGE", "MARITAL_STATUS_IX", "PROFESSION_IX"], outputCol="features")

Next, define estimators you want to use for classification. Random Forest is used in the following example.

In [ ]:
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

Finally, bring indexed labels back to original labels.

In [ ]:
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=stringIndexer_label.labels)

Let's build the pipeline now. A pipeline consists of transformers and an estimator.

In [ ]:
pipeline_rf = Pipeline(stages=[stringIndexer_label, stringIndexer_prof, stringIndexer_gend, stringIndexer_mar, vectorAssembler_features, rf, labelConverter])

Now, you can train your Random Forest model by using the previously defined **pipeline** and **train data**.

In [ ]:
model_rf = pipeline_rf.fit(train_data)

You can check your **model accuracy** now. To evaluate the model, use **test data**.

In [ ]:
predictions = model_rf.transform(test_data)
evaluatorRF = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluatorRF.evaluate(predictions)
print("Accuracy = %g" % accuracy)

Greater accuracy might be achieved using more data, regularization, normalization or other techniques.

<a id="persistence"></a>
## 3. Persist model

In this section you will learn how to store your pipeline and model by using Python client libraries.

First, you must import client libraries.

**Note**: Apache® Spark 2.0 or higher is required.

In [ ]:
from repository.mlrepositoryclient import MLRepositoryClient
from repository.mlrepositoryartifact import MLRepositoryArtifact

Define the service path and create a machine learning repository client.

In [ ]:
service_path = 'https://internal-nginx-svc.ibm-private-cloud.svc.cluster.local:12443'
ml_repository_client = MLRepositoryClient()

Create model artifact (abstraction layer).<br>
<font color="red">**Tip**: Make sure to replace the **XX** in the following code cell with your team number to identify yours from the other teams.</font>

In [ ]:
model_artifact = MLRepositoryArtifact(model_rf, training_data=train_data, name="Product Line Prediction")
model_artifact.meta.add("authorName", "Team XX")

**Tip**: The MLRepositoryArtifact method expects a trained model object, training data, and a model name.

### 3.1: Save pipeline and model

In this subsection you will learn how to save pipeline.

In [ ]:
saved_model = ml_repository_client.models.save(model_artifact)

Get saved model metadata.

**Tip**: Use *meta.available_props()* to get the list of available props.

In [ ]:
saved_model.meta.available_props()

In [ ]:
print "modelType: " + saved_model.meta.prop("modelType")
print "trainingDataSchema: " + str(saved_model.meta.prop("trainingDataSchema"))
print "creationTime: " + str(saved_model.meta.prop("creationTime"))
print "modelVersionHref: " + saved_model.meta.prop("modelVersionHref")
print "label: " + saved_model.meta.prop("label")

**Tip**: **modelVersionHref** is the unique model indentifier.

### 3.2: Load model

In this subsection you will learn how to load back a saved model.

In [ ]:
loadedModelArtifact = ml_repository_client.models.get(saved_model.uid)

You can print for example model name to make sure that model has been loaded correctly.

In [ ]:
print str(loadedModelArtifact.name)

As you can see the name is correct. You have successfully saved and loaded the model.

<a id="visualization"></a>
## 4. Predict locally and visualize

In this section we will score test data using the loaded model and visualize the prediction results with plotly package.

### 4.1: Make local prediction using previously loaded model and test data

In this subsection you will score *predict_data* data set.

In [ ]:
predictions = loadedModelArtifact.model_instance().transform(predict_data)

Preview the results by calling the *show()* method on the predictions DataFrame.

In [ ]:
predictions.show(5)

By tabulating a count, you can see which product line is the most popular.

In [ ]:
predictions.select("predictedLabel").groupBy("predictedLabel").count().show()

### 4.2: Sample visualization of data with Plotly package

In this subsection you will explore prediction results with Plotly, which is an online analytics and data visualization tool.

**Note**: If you experience errors in this next code cell, you may need to install the required packages.  This can be done by issuing the following but only needs to be run one time.<br>
!pip install plotly --user <BR>
!pip install cufflinks==0.8.2 --user
<br><br>
Import Plotly and other required packages.

In [ ]:
import sys
import pandas
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import plotly.graph_objs as go
init_notebook_mode(connected=True)
sys.path.append("".join([os.environ["HOME"]])) 

Convert the Apache Spark DataFrame to a Pandas DataFrame.

In [ ]:
predictions_pdf = predictions.select("prediction", "predictedLabel", "GENDER", "AGE", "PROFESSION", "MARITAL_STATUS").toPandas()

Plot a pie chart that shows the predicted product-line interest.

In [ ]:
cumulative_stats = predictions_pdf.groupby(['predictedLabel']).count()

product_data = [go.Pie(
            labels=cumulative_stats.index,
            values=cumulative_stats['GENDER'],
    )]

product_layout = go.Layout(
    title='Predicted product line client interest distribution',
)

fig = go.Figure(data=product_data, layout=product_layout)
iplot(fig)

With this data set, you might want to do some analysis of the mean AGE per product line by using a bar chart.

In [ ]:
age_data = [go.Bar(
            y=predictions_pdf.groupby(['predictedLabel']).mean()["AGE"],
            x=cumulative_stats.index
            
    )]

age_layout = go.Layout(
    title='Mean AGE per predicted product line',
    xaxis=dict(
        title = "Product Line",
        showline=False,),
    yaxis=dict(
        title = "Mean AGE",
        ),
)

fig = go.Figure(data=age_data, layout=age_layout)
iplot(fig)

Based on bar plot you created, you might make the following conclusion: The mean age for clients that are interested in golf equipment is predicted to be around 50 years old.

<a id="scoring"></a>
## 5. Deploy and score

In this section you will learn how to create online scoring and how to score a new data record. 

The Data Science Experience Local user interface will be used to deploy the model that we previously saved.
<font color="red">Please step through the following steps carefully.</font>

1) Click here https://9.30.106.43:8444 to open another browser tab for DSX Local.<br>
2) Click on the hamburger icon, then click on 'Model Management' and 'Models'.<br>
3) Find the model you saved by looking under 'Creator' for your team name and number.<br>
4) Click on the '...' and select 'Create deployment'<br>
5) On the next screen, enter your team name for 'Name' and click on 'Create'.  Your saved model will now appear under 'Deployments'.<br>
6) Click on your 'Deployment Name' and copy-and-paste the first 'Scoring endpoint'.

This step demonstrates that a REST API call can be used to test the model.

<font color="red">Replace **'<-Scoring endpoint->'** in the next cell with the 'Scoring endpoint' that was previously copy-and-pasted.</font>

In [ ]:
!curl -i -k -X POST <-Scoring endpoint-> -d '{"fields": ["GENDER","AGE","MARITAL_STATUS","PROFESSION"],"records": [["M",23,"Single","Student"]]}' -H "content-type: application/json" 

As we can see we predict that a 23-year-old, single, male student is probably interested in personal accessories (predictedLabel: Personal Accessories, prediction: 1.0). 

This REST call could be leveraged by applications to perform real-time scoring employing a friendly user interface.

<a id="summary"></a>
## 6. Summary and next steps     

 You successfully completed this notebook! You learned how to use Apache Spark machine learning for model creation and deployment. Check out the [Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/pm_service_api_spark.html) for more samples, tutorials, documentation, how-tos, and blog posts. 

### Acknowledgement

This notebook is based on the work of Lukasz Cmielowski, PhD.

Copyright © 2017 IBM. This notebook and its source code are released under the terms of the MIT License.